# Convolutional Neural Network (CNN)

## Introduction

Convolutional neural networks (CNNs or ConvNets) are popular group of neural networks in Deep Learning. 

It was inspired by the the study conducted by Dr.Hubel and Dr.Wisel in 1960’s. They worked on the area of Sensory processing. In this study,they inserted a micro-electrode into the primary visual cortex of an partially anesthetized cat (cat was partially anesthetized so that it would not move during the study) and shown the images to the cat. They took the reading from the electrodes.

The CNN was introduced in 1980 but at that time due to limitation in the computational power of computer it was not popular. A kind of CNN called AlexNet won the [Image Net Chllange in 2012 A.D](http://www.image-net.org/challenges/LSVRC/2012/) then the CNN became very popular in image classification.

## Architecture of Convolutional Neural Network

The Image below shows the architecture of a typical Convolutional Neural Network.

![CNN Architecture](https://upload.wikimedia.org/wikipedia/commons/6/63/Typical_cnn.png)

The image is converted into feature vector. If the image is of size 32X32 with RGB color channel then image. Then the input layer of a typical Neural Network would have 32323 = 3072 weights. This amount still seems manageable, but clearly this fully-connected structure does not scale to larger images. For example, an image of more respectable size, e.g. 200X200X3, would lead to neurons that have 
200X200X3 =120,000 weights. So in Convolution and pooling is done so that we can extratct the important features of a image and size is also reduced. 

A CNN consists of sequence of layers. Each layer has different function. The main layers to build a CNN are 
1. Input 
2. Convolutional Layer 
3. Pooling Layer
4. Fully-Connected Layer

### Input Layer

It holds the raw pixel values of the image. If we have an image of size 32X32 then widthof image=32, height of image=32, and  three color channels Red,Green,Blue.

### Convolutional Layer

In this layer we use kernals to perform the convulution operation. Convolution is the first layer to extract high-level features from an input image. Convolution of an image with different filters can perform operations such as edge detection, blur and sharpen by applying filters.

### Pooling Layer
Pooling is carried out in order to reduce the dimensionality (size). As we see in the diagram above the feature map is getting smaller and smaller. It is done via pooling operation. 

### Fully Connected Layer
A fully connected layer is the plain old neural network. This layer computes the class scores. As with ordinary Neural Networks and as the name implies, each neuron in this layer will be connected to all the numbers in the previous volume.

**A typical CNN consist of one input layer, multiple sequence of convolutional and pooling layer and one fully connected layer.**

In [ ]:
import os
import cv2
import zipfile
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
import matplotlib.pyplot as plt

Extracting the Files From zipped floders

In [ ]:
with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".")

**Taking a Look at some of the Sample images of Dogs from the Dataset**

In [ ]:

# define location of dataset
folder = '/kaggle/working/train/'
for i in range(9):
    # define subplot
    plt.subplot(330 + 1 + i)
    # define filename
    filename = folder + 'dog.' + str(i) + '.jpg'
    # load image pixels
    image = plt.imread(filename)
    # plot raw pixel data
    plt.imshow(image)
    #show the figure
plt.show()

**Taking a Look at some of the Sample images of Cats from the Dataset**

In [ ]:
for i in range(9):
    # define subplot
    plt.subplot(330 + 1 + i)
    # define filename
    filename = folder + 'cat.' + str(i) + '.jpg'
    # load image pixels
    image = plt.imread(filename)
    # plot raw pixel data
    plt.imshow(image)
    #show the figure
plt.show()

**Setting the Constants that will be used throught the program**

In [ ]:
TRAIN_DIR = "/kaggle/working/train/"
TEST_DIR="/kaggle/working/test1/"
TRAIN_SIZE = len([name for name in os.listdir(TRAIN_DIR)])
print("Number of training images:", TRAIN_SIZE)

**Setting the Size of the Image as 80X80 and three channels for RGB**

In [ ]:
IMAGE_WIDTH=80
IMAGE_HEIGHT=80
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

**Using the File name to get the category. Using this information to build a dataframe of file name label**

In [ ]:
filenames = os.listdir(TRAIN_DIR)
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category.lower()=='dog':
        categories.append("dog")
    else:
        categories.append("cat")

In [ ]:
df = pd.DataFrame({
    'filename': filenames,
    'label': categories
})

In [ ]:
df['label']=df['label'].astype(str)

In [ ]:
df.head()

**Creating a Bar Graph to Visulize how many samples of each class do we have**

In [ ]:
df['label'].value_counts().plot(kind='bar')
plt.title("Number of Cats and Dogs Sample in the Dataset")
plt.show()

In [ ]:
df['label'].value_counts()

**Train your algorithm on these files and predict the labels for test1.zip (1 = dog, 0 = cat).**

In [ ]:
df['label'] = df['label'].map({"dog":"1","cat":"0"})

In [ ]:
df['label']=df['label'].astype('str')

**Splitting the Data in Training and Testing Set**

In [ ]:
train_df, valid_df = train_test_split(df, test_size=0.2)

**Using Image Generator to create the data for training the model. The image generator transforms the images in various ways for example by rotating the image. This allows us the train the model with images at different angles**

In [ ]:
train_datagen = ImageDataGenerator(    
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255.,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    TRAIN_DIR, 
    x_col='filename',
    y_col='label',
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    class_mode='binary',
)

In [ ]:
valid_generator = test_datagen.flow_from_dataframe(
    valid_df, 
    TRAIN_DIR, 
    x_col='filename',
    y_col='label',
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    class_mode='binary'
)

**Defining the Model Structure. As explained earlier the CNN we build consist of Convolution, Pooling and Fully Connected Layer**

In [ ]:
model = Sequential()

# First Set of Convolution and Pooling Layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second Set of Convolution and Pooling Layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third Set of Convolution and Pooling Layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fully Connected Layer
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid')) # 2 because we have cat and dog classes

In [ ]:
model.summary()

**Visulizing the Model as a Flowchart**

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
    loss='binary_crossentropy',
    metrics = ['accuracy'])

**Training the Model. Adding some important metrices for in the history variable which we can use for visulization later on**

In [ ]:
history = model.fit_generator(train_generator,validation_data=valid_generator,steps_per_epoch=round(TRAIN_SIZE*(1.-0.2)/32),
    validation_steps=round(TRAIN_SIZE*0.2/32),epochs=20,verbose=1)

**Visulizing the Training Metrices on Each Epoch**

In [ ]:
acc = history.history['accuracy']
val_acc = history.history[ 'val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs   = range(len(acc)) # Get number of epochs
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.legend(['Training Accuracy','Validation Accuracy'])
plt.title('Training and validation accuracy')
plt.figure()
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.legend(['Training Loss','Validation Loss'])
plt.title('Training and validation loss')

In [ ]:
test_filenames = os.listdir(TEST_DIR)
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [ ]:

test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    TEST_DIR, 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=32,
    shuffle=False
)

In [ ]:
import numpy as np
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/32))
threshold = 0.5
test_df['category'] = np.where(predict > threshold, 1,0)

In [ ]:
test_df['category'].value_counts()

In [ ]:
submission_df = test_df.copy()
submission_df['id'] = submission_df['filename'].str.split('.').str[0]
submission_df['label'] = submission_df['category']
submission_df.drop(['filename', 'category'], axis=1, inplace=True)
submission_df.to_csv('submission.csv', index=False)